##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Optimización del gráfico de TensorFlow con Grappler

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/graph_optimization"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/graph_optimization.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/graph_optimization.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/graph_optimization.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a> </td>
</table>

## Descripción general

TensorFlow usa gráficos y ejecuciones de forma "eager" para ejecutar cálculos. Un `tf.Graph` contiene un conjunto de objetos `tf.Operation` (operaciones) que representan unidades de cálculo y objetos  `tf.Tensor` que representan las unidades de datos que fluyen entre las operaciones.

Grappler es el sistema de optimización de gráficos predeterminado en el tiempo de ejecución de TensorFlow. Grappler aplica las optimizaciones en el modo de gráfico (dentro de `tf.function`) para mejorar el rendimiento de sus cálculos de TensorFlow mediante la simplificación de gráficos y otras optimizaciones de alto nivel como la incrustación de cuerpos de función para habilitar las optimizaciones en el procedimiento. Al optimizar el `tf.Graph` también se reduce el uso máximo de la memoria y mejora el uso de hardware al optimizar el mapeo de los nodos de los gráficos para calcular los recursos.

Use `tf.config.optimizer.set_experimental_options()` para tener un control más específico de sus optimizaciones de  `tf.Graph`.


## Optimizadores de gráficos disponibles

Grappler realiza las optimizaciones de gráfico mediante un controlador de nivel superior llamado `MetaOptimizer`. Los siguientes optimizadores de gráficos están disponibles con TensorFlow:

- *Optimizador de plegado continuo*: Infiere estadísticamente el valor de los tensores, siempre que sea posible, al plegar de forma continua los nodos en el gráfico y materializa el resultado con constantes.
- *Optimizador aritmético*: Simplifica las operaciones aritméticas mediante la eliminación de subexpresiones y la simplificación de las declaraciones aritméticas.
- *Optimizador de diseño*: Optimiza el diseño del tensor para ejecutar operaciones que dependen del formato de los datos de forma más eficiente, como las convoluciones.
- *Optimizador de reasignación*: Reasigna subgráficos en implementaciones más eficientes al reemplazar los subgráficos que ocurren comúnmente con kernels monolíticos fusionados y optimizados.
- *Optimizador de memoria*: Analiza el gráfico para inspeccionar el uso máximo de la memoria para cada operación e inserta las operaciones de copia de la memoria CPU-GPU para intercambiar la memoria GPU por la CPU para reducir el uso máximo de memoria.
- *Optimizador de dependencias*: Elimina o reorganiza las dependencias de control para acortar la ruta de acceso crítica para un paso del modelo o habilita otras optimizaciones. También elimina los nodos que nos son efectivamente operaciones, como Identidad.
- *Optimizador de eliminación de entradas*: Elimina los nodos que no afectan a la salida de los gráficos. Suele ejecutarse primero para reducir el tamaño del gráfico y acelerar el proceso en otras pasadas de Grappler.
- *Optimizador de función*: Optimiza la biblioteca de funciones de un programa de TensorFlow e inserta los cuerpos de función para habilitar otras optimizaciones en el procedimiento.
- *Optimizador de forma*: Optimiza los subgráficos que funcionan en la forma y en la información relacionada con la forma.
- *Optimizador de paralelismo automático*: Arregla los gráficos en paralelo al dividirlos por toda la dimensión del lote. Este optimizador está DESACTIVADO de forma predeterminada.
- *Optimizador de bucles*: Optimiza el flujo de control del gráfico al elevar los subgráficos de bucle invariable desde bucles y al remover las operaciones apiladas redundantes en los bucles. También optimiza los bucles con conteos de puntos de intervalos conocidos y elimina las ramas muertas conocidas estadísticamente en condicionales.
- *Optimizador de asignador de ámbito*: Agrega asignadores de ámbito para reducir el movimiento de datos y para consolidar algunas operaciones.
- Marcador para alojar en optimizador/em0}: Intercambia pequeñas operaciones en el CPU. Este optimizador está  DESACTIVADO de forma predeterminada.
- *Optimizador de precisión mixta automática* Convierte los tipos de datos en float16 según corresponda para mejorar el rendimiento. Actualmente, solo aplica para las memorias GPU.
- *Depurador*: Saca los nodos relacionados con las operaciones de depuración como `tf.debugging.check_numerics` y `tf.print` del gráfico. Este optimizador está DESACTIVADO de forma predeterminada.

## Preparación


In [ ]:
import numpy as np
import timeit
import traceback
import contextlib


import tensorflow as tf

Cree un administrador de contexto para alternar entre los estados del optimizador.

In [ ]:
@contextlib.contextmanager
def options(options):
  old_opts = tf.config.optimizer.get_experimental_options()
  tf.config.optimizer.set_experimental_options(options)
  try:
    yield
  finally:
    tf.config.optimizer.set_experimental_options(old_opts)

## Comparar el rendimiento de ejecución con o sin Grappler

TensorFlow 2 y posteriores ejecutan de forma eager de forma predeterminada. Use `tf.function` para cambiar la ejecución predeterminada al modo de gráfico. Grappler se ejecuta automáticamente en segundo plano para aplicar las optimizaciones de gráfico mencionadas anteriormente y mejorar el rendimiento de la ejecución.


### Optimizador de pliegue continuo

Como un ejemplo preliminar, considere una función que realiza operaciones en constantes y devuelve una salida.

In [ ]:
def test_function_1():
  @tf.function
  def simple_function(input_arg):
    print('Tracing!')
    a = tf.constant(np.random.randn(2000,2000), dtype = tf.float32)
    c = a
    for n in range(50):
      c = c@a
    return tf.reduce_mean(c+input_arg)

  return simple_function

Desactive el optimizador de pliegue de constante y ejecute la función:

In [ ]:
with options({'constant_folding': False}):
  print(tf.config.optimizer.get_experimental_options())
  simple_function = test_function_1()
  # Trace once
  x = tf.constant(2.2)
  simple_function(x)
  print("Vanilla execution:", timeit.timeit(lambda: simple_function(x), number = 1), "s")

Active el optimizador de pliegue de constante y vuelva a ejecutar la función para observar como acelera la ejecución de la función.

In [ ]:
with options({'constant_folding': True}):
  print(tf.config.optimizer.get_experimental_options())
  simple_function = test_function_1()
  # Trace once
  x = tf.constant(2.2)
  simple_function(x)
  print("Constant folded execution:", timeit.timeit(lambda: simple_function(x), number = 1), "s")

### Optimizador de depuración

Considere una función simple que verifique el valor numérico de su argumento de entrada y que lo devuelva. 

In [ ]:
def test_function_2():
  @tf.function
  def simple_func(input_arg):
    output = input_arg
    tf.debugging.check_numerics(output, "Bad!")
    return output
  return simple_func

Primero, ejecute la función con el optimizador de depuración desactivado. 

In [ ]:
test_func = test_function_2()
p1 = tf.constant(float('inf'))
try:
  test_func(p1)
except tf.errors.InvalidArgumentError as e:
  traceback.print_exc(limit=2)

`tf.debugging.check_numerics` provoca un error de argumento debido al argumento `Inf` en el `test_func`.

Habilite el optimizador de depuración y vuelva a ejecutar la función. 

In [ ]:
with options({'debug_stripper': True}):
  test_func2 = test_function_2()
  p1 = tf.constant(float('inf'))
  try:
    test_func2(p1)
  except tf.errors.InvalidArgumentError as e:
    traceback.print_exc(limit=2)

El optimizador de depuración saca el nodo `tf.debug.check_numerics` del gráfico y ejecuta la función sin provocar errores. 

## Resumen

El tiempo de ejecución de TensorFlow usa Grappler para optimizar los gráficos automáticamente antes de la ejecución. Use `tf.config.optimizer.set_experimental_options` para habilitar o deshabilitar los distintos optimizadores de gráficos.

Para obtener más información sobre Grappler, consulte <a href="http://web.stanford.edu/class/cs245/slides/TFGraphOptimizationsStanford.pdf" class="external">Optimizaciones de gráfico de TensorFlow</a>.